In [1]:
import os
import json
import openai

from urllib import response

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
def generate_with_gpt3(prompt, max_tokens=100, temperature=1.0, top_p=1.0, num_return_sequences=1):
    responses = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        suffix=None,
        max_tokens=max_tokens,
        temperature=temperature,
        logprobs=None,
        top_p=top_p,
        n=num_return_sequences,
    )["choices"]
    
    return [r["text"] for r in responses]

In [4]:
generate_with_gpt3("Quick question: What is six times seven?", temperature=0.9, num_return_sequences=3)

['\n\n42', '\n\n42', '\n\n42']

#### Load GSM8K

In [5]:
SCRATCH = os.environ['SCRATCH']

In [6]:
data_path = os.path.join(SCRATCH, 'grade-school-math/grade_school_math/data/train.jsonl')

In [7]:
train, test = [], []

with open(data_path, 'r') as f:
    for line in f:
        train.append(json.loads(line.strip()))

In [8]:
train[:3]

[{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
  'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'},
 {'question': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?',
  'answer': 'Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.\nWorking 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.\n#### 10'},
 {'question': 'Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?',
  'answer': "In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50.\nBetty's grandparents gave her 15 * 2 = $<<15*2=30>>30.\nThis mea

#### Few-shot Examples

In [9]:
FEW_SHOT_EXAMPLES = [
    {"Q": "There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?",
     "S": "There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6.",
     "A": "The answer is 6."},
    {"Q": "If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?",
     "S": "There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5.",
     "A": "The answer is 5."},
    {"Q": "Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?",
     "S": "Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39.",
     "A": "The answer is 39."},
    {"Q": "Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?",
     "S": "Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8.",
     "A": "The answer is 8."},
    {"Q": "Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?",
     "S": "Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9.",
     "A": "The answer is 9."},
    {"Q": "There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?",
     "S": "There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29.",
     "A": "The answer is 29."},
    {"Q": "Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?",
     "S": "Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls.",
     "A": "The answer is 33."},
    {"Q": "Olivia has $23. She bought five bagels for $3 each. How much money does she have left?",
     "S": "Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8.",
     "A": "The answer is 8."}
]

In [10]:
def build_prompt_with_few_shot_examples(question, n_shot=5):
    # creat context
    examples = ["Question: {}\nAnswer: {}".format(e["Q"], " ".join([e["S"], e["A"]])) for e in FEW_SHOT_EXAMPLES[:n_shot]]
    context = "\n\n".join(examples)
    
    return "\n\n".join([context, "Question: {}\nAnswer:".format(question)])

In [11]:
question = """\
Weng earns $12 an hour for babysitting. \
Yesterday, she just did 50 minutes of babysitting. How much did she earn?"""

print(build_prompt_with_few_shot_examples(question))

Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
Answer: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.

Question: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
Answer: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.

Question: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
Answer: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.

Question: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
Answer: Jason started with 20 lollipops. Then he had 12 

In [12]:
generate_with_gpt3(build_prompt_with_few_shot_examples(question), temperature=0.95, num_return_sequences=10)

[' Weng earns $12 an hour. She worked for 50 minutes. So she earned 50 minutes * $12 per hour = $12 * (50/60) = $10. The answer is $10.',
 ' Weng earns $12 an hour. She babysat for 50 minutes. So she earned 50/60 * 12 = 10. The answer is 10.',
 ' Weng earns $12 per hour. In 50 minutes, she would have earned $12 * (50/60) = $10. The answer is $10.',
 ' Weng earns $12 an hour. She babysat for 50 minutes yesterday. So she earned $12 * (50/60) = $10. The answer is $10.',
 ' She earns $12 an hour. So for 50 minutes, she would earn 12 * 0.83 = $9.96. The answer is $9.96.',
 ' Weng was paid $12 for every hour she babysat. She only did 50 minutes, so she would be paid for one hour. $12 x 1 = $12. The answer is $12.',
 ' If Weng earns $12 an hour, then she must have earned $12 for 50 minutes of babysitting. 50 minutes is 0.83 hours. So she earned $12 * 0.83 = $9.96. The answer is $9.96.',
 ' Weng earns $12 an hour. In 50 minutes, she would have earned $12 * (50/60) = $10. So the answer is $10.'

#### Generate Scratchpads